## Is it a guinea pig, a hamster or something else?

fork from https://www.kaggle.com/code/jhoward/is-it-a-bird-creating-a-model-from-your-own-data

In [ ]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [2]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
try:
    import google.colab
    isColab = True
except:
    isColab = False

if iskaggle:
    !pip install fastai duckduckgo_search

if isColab:
    !pip install -Uqq fastai
    !pip install -Uqq duckduckgo_search

Not running on Colab


## Step 1: Download images of guinea pigs, hamsters and other rodents

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

Our searches seem to be giving reasonable results, so let's grab a few examples of each of "guinea pig", "hamster" and "rat" photos, and save each group of photos to a different folder (I'm also trying to grab a range of lighting conditions here):

In [ ]:
searches = 'guinea pig','hamster','rat'
path = Path('rodents')
from time import sleep

for output_value in searches:
    dest = (path/output_value)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{output_value} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{output_value} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{output_value} shade photo'))
    sleep(10)
    resize_images(path/output_value, max_size=400, dest=path/output_value)

## Step 2: Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=9, nrows=3)

Here what each of the `DataBlock` parameters means:

    blocks=(ImageBlock, CategoryBlock),

The inputs to our model are images, and the outputs are categories (in this case, "bird" or "forest").

    get_items=get_image_files, 

To find all the inputs to our model, run the `get_image_files` function (which returns a list of all image files in a path).

    splitter=RandomSplitter(valid_pct=0.2, seed=42),

Split the data into training and validation sets randomly, using 20% of the data for the validation set.

    get_y=parent_label,

The labels (`y` values) is the name of the `parent` of each file (i.e. the name of the folder they're in, which will be *bird* or *forest*).

    item_tfms=[Resize(192, method='squish')]

Before training, resize each image to 192x192 pixels by "squishing" it (as opposed to cropping it).

Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

`fastai` comes with a helpful `fine_tune()` method which automatically uses best practices for fine tuning a pre-trained model, so we'll use that.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
#learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(6)

## Step 3: Use our model (and build your own!)

download a guinea pig image and see if our model can predict it:

In [ ]:
#download a photo of a guinea pig
guinea_pig_urls = search_images('guinea pig photos', max_images=1)

from fastdownload import download_url
dest = 'guinea_pig.jpg'
download_url(guinea_pig_urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
is_guinea_pig,_,probs = learn.predict(PILImage.create('guinea_pig.jpg'))
print(f"This is a: {is_guinea_pig}.")
print(f"Probability it's a guinea pig: {probs[0]:.4f}")